## **Semana de Data Science**

- Minerando Dados

### Conhecendo a base de dados

Monta o drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Importando as bibliotecas básicas

In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

Carregando a Base de Dados

In [0]:
# carrega o dataset de london
from sklearn.datasets import load_boston
boston = load_boston()

In [0]:
# descrição do dataset
print (boston.DESCR)

In [0]:
# cria um dataframe pandas
data = pd.DataFrame(boston.data, columns=boston.feature_names)

Adicionando a coluna que será nossa variável alvo

In [0]:
# adiciona a variável MEDV
data['MEDV'] = boston.target

In [0]:
# filtra os top 16 maiores registro da coluna MEDV
top16 = data.nlargest(16, 'MEDV').index

In [0]:
# remove os valores listados em top16
data.drop(top16, inplace=True)

**Definindo um Baseline**

- `Uma baseline é importante para ter marcos no projeto`.
- `Permite uma explicação fácil para todos os envolvidos`.
- `É algo que sempre tentaremos ganhar na medida do possível`.

In [0]:
# converte os dados
data.RM = data.RM.astype(int)

In [0]:
data.info()

In [0]:
# estatística descritiva da coluna numero de quartos
data.RM.describe()

In [0]:
# definindo a regra para categorizar os dados
categorias = []

In [0]:
# alimenta a lista categorias
for i in data.RM.iteritems():
  valor = (i[1])
  if valor <= 4:
    categorias.append('Pequeno')
  elif valor < 7:
    categorias.append('Medio')
  else:
    categorias.append('Grande')

In [0]:
# cria a coluna categorias
data['categorias'] = categorias

In [0]:
# imprime a contagem de categorias
data.categorias.value_counts()

In [0]:
# agrupa as categorias e calcula as médias
medias_categorias = data.groupby(by='categorias')['MEDV'].mean()

In [0]:
# visualizando a variável medias_categorias
medias_categorias

In [0]:
# criando o dicionario com chaves medio, grande e pequeno e seus valores
dic_baseline = {'Grande': medias_categorias[0], 'Medio': medias_categorias[1], 'Pequeno': medias_categorias[2]}

In [0]:
# imprime dicionario
dic_baseline

In [0]:
# cria a função retorna baseline
def retorna_baseline(num_quartos):
  if num_quartos <= 4:
    return dic_baseline.get('Pequeno')
  elif num_quartos < 7:
    return dic_baseline.get('Medio')
  else:
    return dic_baseline.get('Grande')

In [0]:
# chama a função retorna baseline
retorna_baseline(7)

In [0]:
for i in data.RM.iteritems():
  n_quartos = i[1]
  print('Número de quartos é: {} , Valor médio: {}'.format(n_quartos,retorna_baseline(n_quartos)))

In [0]:
# imprime as 5 primeiras linhas do dataframe
data.head()

### Trabalhando com a baseline

Preparando os dados

In [0]:
# remove as colunas colineares, variavel alvo e a coluna categorias
X = data.drop(['RAD','TAX','MEDV','DIS','AGE','ZN','categorias'], axis=1)

In [0]:
# variável 'y' é nossa variável alvo
y = data['MEDV']

In [0]:
# Dados na variável X
X.head()

In [0]:
# Variável alvo
y.head()

Separando os conjuntos de treino e teste

In [0]:
# carrega o método train_test_split
from sklearn.model_selection import train_test_split

In [0]:
# divide os dados entre o conjunto de treino e teste, 80% e 20% respectivamente.
# define qualquer valor para o parâmetro random_state.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

In [0]:
# imprime a quantidade de linhas dos conjuntos
print ('X_train: numero de linhas e colunas: {}'.format(X_train.shape))
print ('X_test: numero de linhas e colunas: {}'.format(X_test.shape))
print ('y_train: numero de linhas e colunas: {}'.format(y_train.shape))
print ('y_test: numero de linhas e colunas: {}'.format(y_test.shape))

Modelo de baseline

In [0]:
# imprimindo as 5 primeiras linhas dos dados de teste
X_test.head()

In [0]:
# definindo uma lista de predicoes
predicoes = []

In [0]:
# para cada elemento do teste alimenta a lista predicoes
for i in X_test.RM.iteritems():
  n_quartos = i[1]
  predicoes.append(retorna_baseline(n_quartos))

In [0]:
# imprime as 10 primeiras linhas da lista de predicoes
predicoes[:10]

Atribuindo resultados a um dataframe

-   `Manteremos um log ao longo do projeto`
-   `Permite a evolução dado o baseline`



In [0]:
# cria um dataframe vazio
df_results = pd.DataFrame()

In [0]:
# adiciona a coluna valor_real
df_results['valor_real'] = y_test.values

In [0]:
# cria a coluna valor_predito_baseline com as predicoes
df_results['valor_predito_baseline'] = predicoes

In [0]:
# imprimindo as 10 primeiras linhas do df_results
df_results.head(10)

Plota o valor real e o baseline

In [0]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Valor Predito Baseline'))
# Plota a figura
fig.show()

Avaliação do modelo baseline

In [0]:
# calcula a métrica rmse
from sklearn.metrics import mean_squared_error
from math import sqrt

In [0]:
rmse = (np.sqrt(mean_squared_error(y_test, predicoes)))

In [0]:
# imprime a performance do modelo
print ('Performance do modelo baseline:')
print('\nRMSE é: {} '.format(rmse))

### Machine Learning

#### Regressão Linear

- `Devido a correlação entre algumas features podemos usar uma regressão linear.`
- `Modelo simples e eficiente.`
- `Solução rápida e robusta.`

In [0]:
# carrega o pacote LinearRegression
from sklearn.linear_model import LinearRegression

In [0]:
# cria o objeto do tipo LinearRegression
lin_model = LinearRegression()

In [0]:
# treina o algoritmo de regressão linear
lin_model.fit(X_train, y_train)

In [0]:
# avaliação do modelo nos dados de teste
y_pred = lin_model.predict(X_test)

In [0]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [0]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

In [0]:
# atribui os resultados no dataframe df_results
df_results['valor_predito_reg_linear'] = lin_model.predict(X_test)

In [0]:
df_results.head(10)

In [0]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Baseline'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines',
                         line = dict(color = '#FEBFB3'),
                         name='Valor Predito Regressão Linear'))

# Plota a figura
fig.show()

##### Arvore de decisão

*   `Algoritmo de aprendizado supervisionado`
*   `Explicabilidade do Modelo`
*   `Simples`


In [0]:
# importa o pacote DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

In [0]:
# cria um objeto do tipo DecisionTreeRegressor
regressor = DecisionTreeRegressor()

In [0]:
# treina o algoritmo
regressor.fit(X_train, y_train)

Avaliando o modelo de Árvore de Decisão

In [0]:
# fazendo as previsões
y_pred = regressor.predict(X_test)

In [0]:
# Adicionando o valor do modelo no dataframe df_results
df_results['valor_predito_arvore'] = y_pred

In [0]:
# visualiza o dataframe df_results
df_results.head(10)

In [0]:
import plotly.graph_objects as go

# cria uma figura
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Valor Predito Baseline'))


# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines+markers',
                         name='Valor Predito Reg Liner'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_arvore,
                         mode='lines+markers',
                         name='Valor Predito Arvore'))
# Plota a figura
fig.show()

Avaliação do Modelo

In [0]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [0]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

#### Random Forest

- `Método Ensemble`
- `Mais robustez para os modelos`

In [0]:
# Importa o método RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

In [0]:
# cria o objeto rf_regressor
rf_regressor = RandomForestRegressor()

In [0]:
# treina o algoritmo
rf_regressor.fit(X_train, y_train)

In [0]:
# avaliação do modelo nos dados de teste
y_pred = rf_regressor.predict(X_test)

In [0]:
# adiciona os resultados no dataframe df_results
df_results['valor_predito_random_forest'] = rf_regressor.predict(X_test)

In [0]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [0]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

In [0]:
# plota os resultados dos modelos e o valor real.
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='markers',
                         name='Valor Real'))

# Linha com os dados da baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         line = dict(color = '#FF00FF'),
                         name='Valor da Baseline'))

# Linha com os dados preditos pela arvore
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_arvore,
                         mode='lines',
                         line = dict(color = '#B2FF66'),
                         name='Valor Predito Árvore'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines',
                         line = dict(color = '#17BECF'),
                         name='Valor Predito Regressão Linear'))


# Linha com os dados preditos pela Random Forest
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_random_forest,
                         mode='lines',
                         line = dict(color = '#7F7F7F'),
                         name='Valor Predito Random Forest'))

# Plota a figura
fig.show()

### Tunning do Algoritmo de Machine Learning

**Parâmetros da Random Forest**

- `O parâmetro n_estimators define a quantidade de árvores que serão usadas para criação dos modelos. Quanto maior o número de árvores, maior será o número de modelos no ensemble.`

- `O parâmtro max_deph define a profundidade da árvore, quanto mais profunda a árvore, mais especializado nos dados de treino o nosso modelo estará.`

- `O parâmetro max_features define a quantidade máxima de nós que serão usados para separar os níveis folha da árvore`

Deploy da Aplicação

In [0]:
X['MEDV'] = y

In [0]:
X.head()

In [0]:
X.to_csv('data.csv', index=False)